# Data cleaning

In [ ]:
import pandas as pd
from datetime import datetime

file_path = 'steam_top_5000_games_from_steamcharts.csv'
df = pd.read_csv(file_path)

print("--- Data Info ---")
df.info()

missing_count = df.isnull().sum()

total_rows = len(df)
missing_percentage = (missing_count / total_rows) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(
    by='Missing Percentage (%)',
    ascending=False
)

missing_df['Missing Percentage (%)'] = missing_df['Missing Percentage (%)'].map('{:.2f}'.format)

print("--- Detailed Missing Values Report ---")
print(missing_df)

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4870 entries, 0 to 4869
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   App ID                 4870 non-null   int64  
 1   Name                   4870 non-null   object 
 2   Price (USD)            4045 non-null   float64
 3   Is Free                4870 non-null   bool   
 4   Release Date           4856 non-null   object 
 5   Days Since Release     4756 non-null   float64
 6   Tags                   4870 non-null   object 
 7   Controller Support     4870 non-null   bool   
 8   Steam Deck Support     4870 non-null   bool   
 9   Languages              4870 non-null   object 
 10  Review Summary         4870 non-null   object 
 11  Total Reviews          4870 non-null   int64  
 12  Positive Reviews       4870 non-null   int64  
 13  All-Time Peak Players  4870 non-null   int64  
 14  Avg Players (30 Days)  4870 non-null  

# Price Data Fix

In [ ]:
df.loc[df['Is Free'] == True, 'Price (USD)'] = 0.0

free_games_missing_price = df[(df['Is Free'] == True) & (df['Price (USD)'].isnull())].shape[0]
print(f"Free games with a missing price after fix: {free_games_missing_price}")

paid_games_missing_price = df[(df['Is Free'] == False) & (df['Price (USD)'].isnull())].shape[0]
print(f"Paid games with a missing price: {paid_games_missing_price}")

Free games with a missing price after fix: 0
Paid games with a missing price: 825


# Date Discrepancy Fix

In [ ]:
standard_parse_fails = pd.to_datetime(df['Release Date'], errors='coerce').isnull()
date_string_exists = df['Release Date'].notnull()

problematic_dates = df[standard_parse_fails & date_string_exists]['Release Date'].unique()

print(f"--- Found {len(problematic_dates)} Unique Problematic Date Formats ---")
print("Examples:")
print(problematic_dates[:10])

--- Found 68 Unique Problematic Date Formats ---
Here are some examples:
['Coming soon' '24 Feb, 2010' '4 Jun, 2025' 'To be announced'
 'November 2025' '21 Oct, 2021' '21 Feb, 2024' '2 Feb, 2016'
 '14 Jul, 2014' 'Q4 2025']


In [ ]:
import pandas as pd
import re
from datetime import datetime

# A dictionary to map quarters to a representative date for approximation
quarter_map = {'Q1': 'Feb 1', 'Q2': 'May 1', 'Q3': 'Aug 1', 'Q4': 'Nov 1'}

def final_date_parser(date_str):
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str).strip()
    parsed_date = pd.NaT

    if date_str.lower() in ['coming soon', 'to be announced', 'tba']:
        return pd.NaT

    try:
        parsed_date = pd.to_datetime(date_str)
    except (ValueError, TypeError):
        quarter_match = re.match(r'(Q[1-4])\s*(\d{4})', date_str, re.IGNORECASE)
        if quarter_match:
            quarter, year = quarter_match.groups()
            month_day = quarter_map[quarter.upper()]
            parsed_date = pd.to_datetime(f'{month_day}, {year}')
        else:
            year_match = re.search(r'(\d{4})', date_str)
            if year_match:
                year = year_match.group(1)
                parsed_date = pd.to_datetime(f'June 1, {year}')
    if pd.notna(parsed_date) and parsed_date > datetime.now():
        return pd.NaT
    return parsed_date

print("--- Applying Robust Date Parser to Fix Inconsistencies ---")

parsed_dates_series = df['Release Date'].apply(final_date_parser)

df['Days Since Release'] = (datetime.now() - parsed_dates_series).dt.days

final_missing_dates = parsed_dates_series.isnull().sum()
final_missing_days = df['Days Since Release'].isnull().sum()

print(f"\nVerification Complete:")
print(f"Total invalid/future/missing release dates found: {final_missing_dates}")
print(f"Total missing values in 'Days Since Release':     {final_missing_days}")

if final_missing_dates == final_missing_days:
    print("\nSuccess: The number of missing values in both columns now match.")
else:
    print("\nError: There is still a discrepancy between the columns.")

--- Applying Robust Date Parser to Fix Inconsistencies ---

Verification Complete:
Total invalid/future/missing release dates found: 52
Total missing values in 'Days Since Release':     52

Success: The number of missing values in both columns now match.


# Saving cleaned data

In [ ]:
cleaned_file_path = 'steam_data_cleaned.csv'

print("--- Final Data Types Check ---")
print(df.dtypes)

df.to_csv(cleaned_file_path, index=False, encoding='utf-8')

print(f"\nSuccess: Cleaned data has been successfully saved to '{cleaned_file_path}'")

--- Final Data Types Check ---
App ID                     int64
Name                      object
Price (USD)              float64
Is Free                     bool
Release Date              object
Days Since Release       float64
Tags                      object
Controller Support          bool
Steam Deck Support          bool
Languages                 object
Review Summary            object
Total Reviews              int64
Positive Reviews           int64
All-Time Peak Players      int64
Avg Players (30 Days)    float64
dtype: object

Success: Cleaned data has been successfully saved to 'steam_data_cleaned.csv'


# Feature engineering

### Multiplayer feature

In [ ]:
import pandas as pd

multiplayer_keywords = ['Multiplayer', 'MMO', 'Co-op', 'Online PvP', 'Online Co-Op', 'LAN Co-Op']

pattern = '|'.join(multiplayer_keywords)

df['IsMultiplayer'] = df['Tags'].fillna('').str.contains(pattern, case=False)

print("--- 'IsMultiplayer' Column Creation Complete ---")

print("\nDistribution of multiplayer vs. non-multiplayer games:")
print(df['IsMultiplayer'].value_counts())

def reorder_tags_for_display(tag_string, keywords):
    if not isinstance(tag_string, str):
        return ""
    
    tags = [tag.strip() for tag in tag_string.split(',')]

    multiplayer_tags = [tag for tag in tags if any(kw.lower() in tag.lower() for kw in keywords)]
    other_tags = [tag for tag in tags if tag not in multiplayer_tags]

    reordered_list = multiplayer_tags + other_tags
    return ', '.join(reordered_list)

df_display = df[['Name', 'Tags', 'IsMultiplayer']].head(10).copy()

df_display['Tags'] = df_display['Tags'].apply(lambda x: reorder_tags_for_display(x, multiplayer_keywords))

print("\n--- Sample of the DataFrame with Reordered Tags for Verification ---")
print(df_display)

--- 'IsMultiplayer' Column Creation Complete ---

Distribution of multiplayer vs. non-multiplayer games:
IsMultiplayer
False    2478
True     2392
Name: count, dtype: int64

--- Sample of the DataFrame with Reordered Tags for Verification ---
                              Name  \
0                       Townscaper   
1                  Tormented Souls   
2                            Waven   
3                     Arcadegeddon   
4                    YKMET: Strade   
5  Titan Quest Anniversary Edition   
6     Age of Darkness: Final Stand   
7                           Ostriv   
8                    Summoners War   
9                        The Crust   

                                                Tags  IsMultiplayer  
0  Experimental, Relaxing, Building, Sandbox, Cas...          False  
1  Survival Horror, Horror, Puzzle, Psychological...          False  
2  Multiplayer, Online Co-Op, Asynchronous Multip...           True  
3  Online Co-Op, Multiplayer, Roguelite, Arcade, ...      

### Success score

In [ ]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
from scipy.stats import norm

def get_possible_steam_labels(ratio, total_reviews):
    labels = set()
    
    if total_reviews == 0:
        labels.add('No user reviews')
    elif 1 <= total_reviews < 10:
        labels.add(f'{total_reviews} user review{"s" if total_reviews > 1 else ""}')
        if total_reviews == 1:
            labels.add('1 user reviews')

    if ratio >= 0.95:
        labels.add('Overwhelmingly Positive')
    if ratio >= 0.80:
        labels.add('Very Positive')
        labels.add('Positive')
    elif 0.70 <= ratio < 0.80:
        labels.add('Mostly Positive')
    elif 0.40 <= ratio < 0.70:
        labels.add('Mixed')
    elif 0.20 <= ratio < 0.40:
        labels.add('Mostly Negative')
    elif 0.0 <= ratio < 0.20:
        labels.add('Very Negative')
        labels.add('Overwhelmingly Negative')
    if 0.0 <= ratio <= 0.39:
        labels.add('Negative')
    return labels

df['PositiveReviewRatio'] = (df['Positive Reviews'] / df['Total Reviews']).fillna(0)
df['PossibleLabels'] = df.apply(lambda row: get_possible_steam_labels(row['PositiveReviewRatio'], row['Total Reviews']), axis=1)
df['LabelsMatch'] = df.apply(lambda row: row['Review Summary'] in row['PossibleLabels'], axis=1)

match_counts = df['LabelsMatch'].value_counts()
print("--- Final Review Label Validation Results ---")
print(f"Games where labels MATCH:       {match_counts.get(True, 0)}")
print(f"Games where labels DO NOT MATCH:  {match_counts.get(False, 0)}\n")

def calculate_wilson_score(positive_reviews, total_reviews):
    if total_reviews == 0: return 0
    z = norm.ppf(0.975)
    p_hat = positive_reviews / total_reviews
    return (p_hat + z**2/(2*total_reviews) - z * np.sqrt((p_hat*(1-p_hat)+z**2/(4*total_reviews))/total_reviews))/(1+z**2/total_reviews)

def calculate_commercial_success_score(positive_reviews, total_reviews, wilson_score):
    if total_reviews == 0: return 0
    popularity_bonus = np.log10(total_reviews + 1)
    return wilson_score * popularity_bonus

df['ReviewSuccessScore'] = df.apply(lambda r: calculate_wilson_score(r['Positive Reviews'], r['Total Reviews']), axis=1)
df['CommercialSuccessScore'] = df.apply(lambda r: calculate_commercial_success_score(r['Positive Reviews'], r['Total Reviews'], r['ReviewSuccessScore']), axis=1)

print("--- 'ReviewSuccessScore' and 'CommercialSuccessScore' Columns Created ---")
print("\n--- Top 10 Games by 'CommercialSuccessScore' ---")

top_10_commercial = df.sort_values(by='CommercialSuccessScore', ascending=False)[['Name', 'Total Reviews', 'CommercialSuccessScore']].head(10)
top_10_commercial.index = np.arange(1, len(top_10_commercial) + 1)
print(top_10_commercial)

df = df.drop(columns=['PositiveReviewRatio', 'PossibleLabels', 'LabelsMatch'])

--- Final Review Label Validation Results ---
Games where labels MATCH:       4870
Games where labels DO NOT MATCH:  0

--- 'ReviewSuccessScore' and 'CommercialSuccessScore' Columns Created ---

--- Top 10 Games by 'CommercialSuccessScore' ---
                Name  Total Reviews  CommercialSuccessScore
1           Terraria         564427                5.617196
2        Garry's Mod         505213                5.532539
3     Stardew Valley         362144                5.494969
4    Baldur's Gate 3         416424                5.441643
5       Phasmophobia         335343                5.311951
6     Lethal Company         268576                5.290373
7         ELDEN RING         412554                5.288536
8   Counter-Strike 2        1353359                5.265278
9   Wallpaper Engine         211681                5.225028
10     Left 4 Dead 2         216151                5.200453


### Game longevity score

In [ ]:
import numpy as np
import pandas as pd

def calculate_longevity_score(avg_players, days_since_release):
    if pd.isna(avg_players) or pd.isna(days_since_release) or days_since_release <= 0:
        return 0
    longevity_score = np.log10(avg_players + 1) * np.log10(days_since_release + 1)
    return longevity_score

df['LongevityScore'] = df.apply(
    lambda row: calculate_longevity_score(row['Avg Players (30 Days)'], row['Days Since Release']),
    axis=1
)

print("--- 'LongevityScore' Column Creation Complete ---")
print("\n--- Top 10 Games by Longevity Score ---")

top_10_longevity = df.sort_values(by='LongevityScore', ascending=False)[['Name', 'LongevityScore']].head(20)
top_10_longevity['LongevityScore'] = top_10_longevity['LongevityScore'].map('{:.4f}'.format)
top_10_longevity.index = np.arange(1, len(top_10_longevity) + 1)
print(top_10_longevity)

--- 'LongevityScore' Column Creation Complete ---

--- Top 10 Games by Longevity Score ---
                                 Name LongevityScore
1                    Counter-Strike 2        22.7456
2                              Dota 2        21.5048
3                 PUBG: BATTLEGROUNDS        20.0921
4                     Team Fortress 2        18.2761
5              Counter-Strike: Source        18.2362
6                      Stardew Valley        17.6734
7           Grand Theft Auto V Legacy        17.6082
8                         War Thunder        17.5635
9                                Rust        17.4692
10                           Warframe        17.2968
11                   Wallpaper Engine        17.2199
12  Tom Clancy's Rainbow Six® Siege X        17.1920
13                      Hollow Knight        17.1897
14                       No Man's Sky        17.1806
15                      Apex Legends™        17.0689
16             Euro Truck Simulator 2        16.8711
17      

### Language market score

In [ ]:
import numpy as np
import pandas as pd

language_popularity = {
    'English': 35.62, 'Simplified Chinese': 26.03, 'Russian': 9.09,
    'Spanish - Spain': 4.11, 'Portuguese - Brazil': 3.77, 'Korean': 3.59,
    'German': 2.83, 'Japanese': 2.82, 'French': 2.29, 'Polish': 1.6,
    'Traditional Chinese': 1.37, 'Turkish': 1.19, 'Thai': 0.94,
    'Ukrainian': 0.69, 'Spanish - Latin America': 0.65, 'Italian': 0.6,
    'Czech': 0.54, 'Hungarian': 0.37, 'Portuguese - Portugal': 0.34,
    'Dutch': 0.26, 'Swedish': 0.26, 'Danish': 0.22, 'Vietnamese': 0.2,
    'Finnish': 0.15, 'Indonesian': 0.13, 'Norwegian': 0.12,
    'Romanian': 0.11, 'Greek': 0.06, 'Bulgarian': 0.04, 'Arabic': 0.00
}

def calculate_language_score(language_string, popularity_data):
    if pd.isna(language_string):
        return 0
    
    supported_languages = [lang.strip().lower() for lang in language_string.split(',')]
    
    total_score = 0
    matched_keys = set()
    
    for supported_lang in supported_languages:
        for key, value in popularity_data.items():
            if supported_lang in key.lower() and key not in matched_keys:
                total_score += value
                matched_keys.add(key)
                
    return total_score

df['LanguageMarketScore'] = df['Languages'].apply(
    lambda x: calculate_language_score(x, language_popularity)
)

print("--- 'LanguageMarketScore' Column Creation Complete ---")
print("\n--- Top 10 Games by Language Market Score ---")

top_10_language = df.sort_values(by='LanguageMarketScore', ascending=False)[['Name', 'LanguageMarketScore']].head(10)
top_10_language['LanguageMarketScore'] = top_10_language['LanguageMarketScore'].map('{:.2f}%'.format)
top_10_language.index = np.arange(1, len(top_10_language) + 1)
print(top_10_language)

--- 'LanguageMarketScore' Column Creation Complete ---

--- Top 10 Games by Language Market Score ---
                      Name LanguageMarketScore
1             Auto Clicker              99.99%
2         Counter-Strike 2              99.99%
3    Tap Ninja - Idle game              99.99%
4   Soccer Online: Ball 3D              99.99%
5        iTop Easy Desktop              99.99%
6       Nine-Ball Roulette              99.99%
7           The Classrooms              99.99%
8           Wallpaper Play              99.99%
9              Your Mother              99.99%
10                  BOKURA              99.99%


# Final cleaned data table with new features

In [ ]:
columns_to_drop = [
    'Is Free',
    'Review Summary'
]
df_final = df.drop(columns=columns_to_drop).copy()

print("--- Final DataFrame Structure ---")
print(f"The following columns have been dropped: {columns_to_drop}")
print("\nFinal DataFrame Info:")
df_final.info()

final_file_path = 'steam_data_final.csv'
df_final.to_csv(final_file_path, index=False, encoding='utf-8')

print(f"\nFeature engineering complete. The final dataset has been saved to '{final_file_path}'")

--- Final DataFrame Structure ---
The following columns have been dropped: ['Is Free', 'Review Summary']

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4870 entries, 0 to 4869
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   App ID                  4870 non-null   int64  
 1   Name                    4870 non-null   object 
 2   Price (USD)             4045 non-null   float64
 3   Release Date            4856 non-null   object 
 4   Days Since Release      4818 non-null   float64
 5   Tags                    4870 non-null   object 
 6   Controller Support      4870 non-null   bool   
 7   Steam Deck Support      4870 non-null   bool   
 8   Languages               4870 non-null   object 
 9   Total Reviews           4870 non-null   int64  
 10  Positive Reviews        4870 non-null   int64  
 11  All-Time Peak Players   4870 non-null   int64  
 12  Avg Players (30 Da